<a href="https://colab.research.google.com/github/NonKhuna/JobTopGun-job-title-classification/blob/main/bert_jobtopgun_new_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [ ]:
!gdown 1o_Bu3xD8CCNFqjJ7mqVRfcCV49Zpe8NY
!unzip /content/jobtopgun-job-title-classification.zip

Downloading...
From: https://drive.google.com/uc?id=1o_Bu3xD8CCNFqjJ7mqVRfcCV49Zpe8NY
To: /content/jobtopgun-job-title-classification.zip
100% 249k/249k [00:00<00:00, 93.6MB/s]
Archive:  /content/jobtopgun-job-title-classification.zip
  inflating: occupation_group_mapper.csv  
  inflating: occupation_mapper.csv   
  inflating: sample_for_submission.csv  
  inflating: test_for_submission.csv  
  inflating: train.csv               


In [ ]:
! pip install --quiet pythainlp
! pip -q install tqdm
! pip -q install simpletransformers
! pip install marisa_trie
! pip install autocorrect 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd 

from torch.utils.data import Dataset
from IPython.display import display
from pythainlp.tokenize import word_tokenize
from collections import defaultdict
from sklearn.metrics import accuracy_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

np.random.seed(23)

# Prepare Data

In [ ]:
df_dataset = pd.read_csv("train.csv")
df_occ_map = pd.read_csv("occupation_mapper.csv")
df_occ_g_map = pd.read_csv("occupation_group_mapper.csv")
df_dataset.tail()

,job_title,occupation_group,occupation,occupation_group_index,occupation_index
7995,พนักงานพัฒนาสังคมชุมชน,Social Worker,Social Worker,137,519
7996,Head of Marketing,Marketing,Marketing Director,88,342
7997,Recruitment &Training<br>(2 Positions),HR,Recruiter,69,477
7998,แพทย์ผิวหนัง Full Time,Physician/Doctor,Dermatologist,110,164
7999,Visual Display Designer,Display Designer,Display Designer,42,171


In [ ]:
# Drop un-use columns and update field name
try:
    df_dataset.drop(['occupation_group', 'occupation'], axis=1, inplace=True)
    df_dataset.columns = ['input', 'occ_g', 'occ']
    print("updated")
except:
    pass

# clean data
df_dataset.tail()

updated


,input,occ_g,occ
7995,พนักงานพัฒนาสังคมชุมชน,137,519
7996,Head of Marketing,88,342
7997,Recruitment &Training<br>(2 Positions),69,477
7998,แพทย์ผิวหนัง Full Time,110,164
7999,Visual Display Designer,42,171


In [ ]:
df_dataset['occ'].value_counts().describe()

count    597.000000
mean      13.400335
std       12.645351
min        3.000000
25%        8.000000
50%       10.000000
75%       15.000000
max      207.000000
Name: occ, dtype: float64

In [ ]:
# fix imbalance data : 15 sample each category

# index_i = 
# df_dataset[df_dataset['occ'] == 50].sample(n=20, random_state=1, replace=True)

## Find word effect

In [ ]:
# Count word affect
def count_word_effect(df_dataset, trie) :
  word_effect = defaultdict(list)
  for i in range(len(df_dataset)) :
    sents = df_dataset.iloc[i,0].lower()
    label = df_dataset.iloc[i,2]
    sent_normalize = normalize(sents)
    text_list = word_tokenize(sent_normalize, custom_dict=trie)
    new_text_list = []
    for t in text_list:
        # if t in stopwords: continue
        # if t in trie : continue
        word_effect[t].append((i,label))
  word_effect_map = defaultdict(lambda: (-1,-1,-1))
  for x in word_effect :
    uniqu_row = len(set([occ[1] for occ in word_effect[x]]))
    total = len(word_effect[x])
    prop = uniqu_row/ total
    word_effect_map[x] = (uniqu_row, total, prop)
  return word_effect_map


In [ ]:
# load province created by myself
!gdown 1L1Sh4sUQVDTMQTUl4ROuzdhrS6SYHyfY # province
!gdown 15Rpfm-Fdk40pqIrUDwmnt_kbpXNMQoRt # bkk_area


Downloading...
From: https://drive.google.com/uc?id=1L1Sh4sUQVDTMQTUl4ROuzdhrS6SYHyfY
To: /content/province_th_eg.json
100% 4.20k/4.20k [00:00<00:00, 7.03MB/s]
Downloading...
From: https://drive.google.com/uc?id=15Rpfm-Fdk40pqIrUDwmnt_kbpXNMQoRt
To: /content/bkk_area.json
100% 2.04k/2.04k [00:00<00:00, 3.73MB/s]


In [ ]:
import re
import string 
import json

from pythainlp.util import normalize
from pythainlp.corpus import thai_stopwords
from nltk.corpus import stopwords as stopwords_corpus

from pythainlp.spell import correct
from pythainlp.corpus.common import thai_words
import marisa_trie

from autocorrect import Speller


"""
current feature
- custom dict for specific name or word
- word_effect_map: amount of time that a word appear in different data
- english spell for correct the word 
- thai spell for correct thai word
- remove province
- remove information about date
- change some informal word ex. ผจก
- remove working time ex. full-time , part-time
- remove_websyntax
- remove number
- add stop word about position
"""

# Out of dict
ood = defaultdict(int)

# province, need to load from `!gdown 1f4mfjuNyV_Gdu6ABditbzF-64UvC_vQz`
thai_province = []
with open("/content/province_th_eg.json", 'r') as f:
  thai_province = json.load(f)['data'] #need to be lower before
  thai_province.append('samutprakarn')

# area in bangkok
bkk_area = []
with open("/content/bkk_area.json") as f:
   bkk_area = json.load(f)['data']

# custom dictionary
new_dict = set(thai_words())
new_word = ['ซีรี่', "3d","2d", "skincare", "sale", 'sales', 'เดรดิต' , "petro", 'อารีย์' , "ปฎิบัติการ", "เวชสำอาง", "สลิท", "ธุรการ", "พนักงาน", "ไฮดรอลิค", "บอยเลอร์" , "ผู้จัดการเขต", "ผู้จัดการสาขา", "welder", "commis"] + bkk_area
for word in new_word:
    new_dict.add(word) # for 3d
trie = marisa_trie.Trie(new_dict)
extra_char = r'[”“’–‘ฯ]'

# get word_effect
word_effect_map =  count_word_effect(df_dataset, trie)
minimum_require_to_correct = 3 # use with word_effect_map that require target correct should more than current and minium uniq current =  minimum_require_to_correct

# spell eng checker
spell = Speller(lang='en')

def check_eng(sent) :
    return re.match(r"[a-zA-Z][a-zA-Z0-9]*", sent)

def remove_address(msg, thai_province) :
    """ Remove province since it's not help to classify"""
    # msg = re.sub(r"สาขา(?!ต่างประเทศ)[ก-๏]+", " ", msg)
    msg = re.sub(r'ปฏิบัติงานที่', " ", msg)

    msg = re.sub(r'(bts|mrt)[ ]?[ก-๏a-z]*', " ", msg)
    # msg = re.sub(r'(bts|mrt)', " ", msg)

    msg = re.sub(r"[จอ]\.[ก-๏]+", " ", msg)
    # msg = re.sub(r'[จอ]\.', " ", msg)

    area = ['phuket','สาทร', 'rayong', "หนองคาย", " area ", 'siam', 'icon', 'กทม', 'จป', 'นครนายก','บางนา', 'sukhumvit' "เมกาบางนา", "bkk", 'pattaya','พัทยา','จอมเทียน','พระราม', 'ปทุม','อยุธยา', 'หัวหิน', 'rangsit']
    thai_area = ['(ภาค)?กลาง', '(ภาค)?ตะวันออก(เฉียงเหนือ)?', '(ภาค)?ใต้', '(ภาค)?เหนือ', 'อีสาน']
    thai_province_regx = "(" + "|".join(thai_area + area) + ")"
    msg = re.sub(thai_province_regx, " ", msg)
    return msg

def remove_date(msg):
    day = ["อาทิตย์","จันทร์","อังคาร","พุธ","พฤหัสบดี","ศุกร์","เสาร์"]
    period = ['month','months','yrs','year','years', 'ปี']
    day_regx = "(" + "|".join(day+period) + ")"
    msg = re.sub(day_regx, " ", msg)
    return msg

def change_formal_word(msg):
    set_word = {"ดราฟท์":"เขียนแบบ", "ผจก.":"ผู้จัดการ", 'จนท.':"เจ้าหน้าที่", "r&d":"วิจัยและพัฒนา", '(เเครชเชียร์|แคชเชียร์|แคสเชียร์)': "cashier", "(เสริฟ์|เสริฟ)": "serve", 'รปภ.':" รักษาความปลอดภัย ", 'f&b':"fb"}  
    for w in set_word.items():
        msg = re.sub(w[0] , w[1], msg)
    return msg
  
def delete_specific_name(msg): 
    words = ['krungsri',"กรุงศรีอยุธยา","dtac", "โลตัส"]  # ธนาคาร 'big c', 'scb', 'teparuk plant', 'บริษัทในเครือ'
    uniq_regx = "(" + "|".join(words) + ")" 
    msg = re.sub(uniq_regx, " ", msg)
    return msg

def remove_workingtime(msg):
    msg = re.sub(r"(full|part).*time",' ',msg)
    msg = re.sub(r"(ยินดี[ก-๏]*จบใหม่|จบใหม่|(new )?grad)", ' ', msg)
    return msg

def remove_websyntax(msg) :
    msg = re.sub(r'<.*?>',' ', msg)
    msg = re.sub(r'#',' ',msg)
    return msg

def clean_msg(msg):
    msg = msg.lower().strip()
    msg = remove_websyntax(msg) 
    msg = change_formal_word(msg)
    msg = remove_address(msg, thai_province)
    msg = remove_date(msg)
    # msg = delete_specific_name(msg)
    msg = re.sub(r'(?![23][d])\d+k?',' ',msg) # remove number except 2D and 3D
    msg = remove_workingtime(msg)
    
    msg = re.sub(extra_char, ' ', msg)
    for c in '!"#$%&\'*+,-./;<=>?@[\]^_`{|}~': # () :
        msg = re.sub(r'\{}'.format(c),' ',msg) 
    msg = ' '.join(msg.split())
    return msg

def custom_word_tokenize(sent):
    sent_normalize = normalize(sent)
    text_list = word_tokenize(sent_normalize, custom_dict=trie)
    new_text_list = []
    for t in text_list:
        if t in stopwords: continue
        if t in thai_province : continue
        if t in trie :
            new_text_list.append(t)
            continue
        ood[t] += 1
        if check_eng(t) :
            correct_t = spell(t)
            if t != correct_t and word_effect_map[t][0] < minimum_require_to_correct and word_effect_map[correct_t][0] >= word_effect_map[t][0]: 
              # print(t, correct_t, word_effect_map[t], word_effect_map[correct_t])
              new_text_list.append(correct_t)
            else :
              new_text_list.append(t)
        else :
            correct_t = correct(t)
            if t != correct_t and word_effect_map[t][0] < minimum_require_to_correct and word_effect_map[correct_t][0] >= word_effect_map[t][0]: 
              # print(f"correct form {t} to {correct_t}, original {text_list}")
              new_text_list.append(correct_t)
            else :
              new_text_list.append(t)
    return new_text_list

def custom_tokenize(sent):
    return custom_word_tokenize(clean_msg(sent))

stopwords = list(thai_stopwords()) + list(stopwords_corpus.words('english'))
# stopwords = list(thai_stopwords())
stopwords.remove('it')
stopwords.append("")
stopwords.append(" ")

# position stop word
word_position = ['based', 'base', 'ประจำ',"interview", 'exp', 'experienced', 'experience' , 'สัมภาษณ์', 'รายได้',
                 'ตำแหน่ง','อัตรา','position',"positions" ,"urgent", "ด่วน", "urgently", "required", "รับสมัคร" ,'เขต','เปิดรับ', 'สาขา',
                 'welcome','welcomes', 'many'] # asst "แผนก" "หน่วยงาน"
# word_position = word_position + ["senior",'sr','junior','jr']
# word_position = word_position =  ['permanent', 'temporary','contract', 'งานประจำ', "สัญญาจ้าง"]
for word in word_position :
    stopwords.append(word)



ex = """ 3d Animator “ ’ ” - IT Support (ประจำสาขาต่างประเทศ) 20-50K Petrochemical Sales BTS อารีย์, BTS พระโขนง 
"""

clean_ex = clean_msg(ex)
clean_ex , custom_word_tokenize(clean_ex)

('3d animator it support (ประจำสาขาต่างประเทศ) petrochemical sales',
 ['3d',
  'animator',
  'it',
  'support',
  '(',
  'ต่างประเทศ',
  ')',
  'petro',
  'chemical',
  'sales'])

## For examine

In [ ]:
# word_effect = list(word_effect_map.items())
# sorted(word_effect, key=lambda x: x[1][0], reverse=True)

In [ ]:
df_dataset['clean_input'] = df_dataset['input'].apply(lambda x: custom_word_tokenize(clean_msg(x)))

In [ ]:
df_dataset[df_dataset['occ'] == 1][['input','clean_input']]

,input,clean_input
774,ผู้อำนวยการบริหารงานลูกค้า ด่วน,"[ผู้อำนวยการ, บริหารงาน, ลูกค้า]"
836,Account Director,"[account, director]"
1943,ผู้อำนวยการบริหารงานลูกค้า,"[ผู้อำนวยการ, บริหารงาน, ลูกค้า]"
2263,Account Director,"[account, director]"
3549,Account Director - Urgent,"[account, director]"
3875,Group Account Director,"[group, account, director]"
4141,ผู้อำนวยการบริหารงานลูกค้า,"[ผู้อำนวยการ, บริหารงาน, ลูกค้า]"
5085,Account Director,"[account, director]"
7781,Assistant Account Director (1 Position) /รองผู...,"[assistant, account, director, (, ), รองผู้อำน..."


In [ ]:
def find_target_example(text, sent):
    bools = []
    for x in sent:
        if type(x) == str and text in x.lower() :
            bools.append(True)
        elif type(x) == list and text in x:
            bools.append(True)
        else :
            bools.append(False)
    print(sum(bools))
    return bools

text = "phyathai".lower()
df_dataset[find_target_example(text, df_dataset['input'])][['input','clean_input','occ']]
# df_dataset[find_target_example(text, df_dataset['clean_input'])]

1


,input,clean_input,occ
6704,Database Website Senior/Manager <br>( Phyatha...,"[database, website, senior, manager, (, phyath...",536


In [ ]:
text = "โปรแจ็ก".lower()
df_dataset[find_target_example(text, df_dataset['input'])]
# df_dataset[find_target_example(text, df_dataset['clean_input'])]

1


,input,occ_g,occ,clean_input
7194,โปรแจ็กเซล,132,495,"[โปร, แจ็ก, เซ, ล]"


In [ ]:
# df_dataset[df_dataset['input'].duplicated()].groupby('input').count()

## Gen data

In [ ]:
# df_unique_dataset = df_dataset.drop_duplicates(subset=['input'], keep='first')
# # fix occ = 210 have only 1 example
# df1 = pd.DataFrame({
#     "input": ['ผู้บริหารระดับสูง'],
#     "occ_g": [142],
#     'occ': [210],
#     'clean_input': [["ผู้บริหารระดับสูง"]]
# })
# df_unique_dataset = df_unique_dataset.append(df1)

# df_unique_dataset = df_unique_dataset.reset_index()

In [ ]:
# Splitting Data 
from sklearn.model_selection import train_test_split
X = df_dataset[['input']]
y = df_dataset[['occ']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
# X_train, X_val, y_train, y_val = train_test_split( X_train, y_train, test_size=1/9, random_state=42, stratify=y_train)

train = X_train.join(y_train)
train = df_dataset
test = X_test.join(y_test)
# val = X_val.join(y_val)

train.shape, test.shape

((8000, 4), (800, 2))

In [ ]:
# # Up sampling & Down sampling of text data using SMOTE
# from imblearn.over_sampling import SMOTE
# from sklearn.preprocessing import LabelBinarizer
# sm = SMOTE(random_state = 42, k_neighbors=2)
# lb = LabelBinarizer()

# X_train = lb.fit_transform(X_train)
# X_train_bin, y_train = sm.fit_resample(X_train, y_train)
# X_train = lb.inverse_transform(X_train_bin)
# train = X_train.join(y_train)
# train.shape

In [ ]:
def input_pipeline(sent):
    return " ".join(custom_word_tokenize(clean_msg(sent)))
train['wordseg_input'] = train['input'].apply(input_pipeline)
test['wordseg_input'] = test['input'].apply(input_pipeline)
# val['wordseg_input'] = val['input'].apply(input_pipeline)

In [ ]:
train_ = pd.DataFrame({"text": train["wordseg_input"], "labels": train["occ"]})
test_ = pd.DataFrame({"text": test["wordseg_input"], "labels": test["occ"]})
# val_ = pd.DataFrame({"text": val["wordseg_input"], "labels": val["occ"]})
train_.labels = pd.Categorical(train_.labels)
test_.labels = pd.Categorical(test_.labels)
# val_.labels = pd.Categorical(val_.labels)
train_

,text,labels
0,ผู้แทน ฝ่ายบริหาร,211
1,เภสัชกร แหลมฉบัง,413
2,ครูผู้สอน วิชา สุขศึกษา,276
3,atc,23
4,energy monitoring engineer,195
...,...,...
7995,พนักงาน พัฒนา สังคม ชุมชน,519
7996,head marketing,342
7997,recruitment training ( ),477
7998,แพทย์ ผิวหนัง,164


# Train Model

In [ ]:
# # upload model before !|
# !gdown --fuzzy 1nuYPHtgNzKfUmgznuc_k1Bv2M8L6pi8-
# !jar xvf /content/checkpoint-4000.zip

In [ ]:
# model
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import sklearn
import math

name = "Geotrend/bert-base-en-th-cased"
# name = "/content/outputs/checkpoint-4000"
# name = "airesearch/wangchanberta-base-att-spm-uncased"
num_labels = len(df_occ_map)

BATCH_SIZE = 64
SAVE_EVERY_N_EPOCHS = 2
# steps_per_epoch = math.floor(len(train_) / SAVE_EVERY_N_EPOCHS)
# if(len(train_) % SAVE_EVERY_N_EPOCHS > 0):
#     steps_per_epoch +=1

model_args = {
    "reprocess_input_data": True,
    'train_batch_size':BATCH_SIZE,
    "use_early_stopping": True, #ถ้าค่า lose น้อยลงเกิน 0.05 จะตัด 
    "early_stopping_delta": 0.01,
    "early_stopping_metric": "mcc",
    "early_stopping_metric_minimize": False,
    "early_stopping_patience": 5,
    "evaluate_during_training_steps": 500,
    "fp16": False,
    "overwrite_output_dir":True,
    'use_cached_eval_features' : False, 
    'max_seq_length': 32,
    'no_cache': True,
    'manual_seed' : 24,
    # 'learning_rate': 1e-5,
    "save_model_every_epoch": False,
    'best_model_dir': "/content/bestModel",
    "num_train_epochs":25,
    'custom_parameter_groups':[{
        "params": ["classifier.weight", "bert.encoder.layer.10.output.dense.weight"],
        "lr": 1e-3,
    }]
}

model = ClassificationModel(
     "bert", 
     name,
     use_cuda=torch.cuda.is_available(),
     args=model_args,  
     num_labels=num_labels,
)
print(torch.cuda.is_available())


# model.train_model(train_, show_running_loss = True, eval_df=test_, acc=sklearn.metrics.accuracy_score)

Some weights of the model checkpoint at Geotrend/bert-base-en-th-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Geotrend/bert-

True


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29151, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torchinfo
torchinfo.summary(model.model)

Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              22,387,968
│    │    └─Embedding: 3-2                              393,216
│    │    └─Embedding: 3-3                              1,536
│    │    └─LayerNorm: 3-4                              1,536
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             85,054,464
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 590,592
│    │    └─Tanh: 3-8                                   --
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           459,093
Total params: 

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_)
result

  0%|          | 0/800 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

{'mcc': 0.9649227891694391, 'eval_loss': 0.07044900257867993}

In [ ]:
from sklearn.metrics import classification_report
# model_best = ClassificationModel(
#      "bert", 
#      "/content/outputs/checkpoint-8000",
#      use_cuda=torch.cuda.is_available(),
#      args=model_args,  
#      num_labels=num_labels,
# )
result, model_outputs, wrong_predictions = model.eval_model(train_)
preds =model_outputs.argmax(axis = 1)
print(classification_report(train_['labels'], preds))

  0%|          | 0/8000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1000 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         9
           2       0.93      0.96      0.95        28
           3       0.95      1.00      0.97        19
           4       0.90      0.96      0.93        48
           5       0.90      1.00      0.95         9
           6       1.00      0.88      0.93         8
           7       1.00      1.00      1.00         6
           8       0.96      0.91      0.93       127
           9       1.00      1.00      1.00         6
          10       0.75      1.00      0.86         6
          11       1.00      0.60      0.75        10
          12       1.00      0.93      0.96        14
          13       1.00      0.91      0.95        11
          14       1.00      1.00      1.00         9
          15       0.89      1.00      0.94         8
          16       1.00      1.00      1.00         4
          17       1.00    

In [ ]:
# Load the best model

df_test = pd.read_csv("test_for_submission.csv")
df_test.columns = ['Id', "input"]
X_submit_test = list(df_test['input'].apply(input_pipeline))

y_pred = model.predict(X_submit_test)
df_test['Predicted'] = y_pred[0]
df_test.head()
out = df_test[['Id', "Predicted"]]
out.set_index("Id",inplace=True)
out.to_csv("submission.csv")
out

  0%|          | 0/2798 [00:00<?, ?it/s]

  0%|          | 0/350 [00:00<?, ?it/s]

,Predicted
Id,
0,390
1,552
2,201
3,400
4,96
...,...
2793,343
2794,225
2795,429


In [ ]:
from google.colab import files
files.download('submission.csv') 